#### Loading packages and database

In [ ]:
library(caret) #Caret is a complete machine learning solution
library(here)
source(here("modules","bib.r"))
source(here("modules","data_cleaning.r"))
source(here("modules","estimadistribuicao.R"))

Regressões lineares e ridget 

Todos os modelos de regresão são GLM / LM ( glm e lm são usados para ajustar modelos lineares generalizados, ambos iram produzir exatamente os mesmos resultados). A família  estatística usada para ajustar o modelo:  gaussiana, mas outras opções incluem binomial, Gamma e poisson, entre outras.

Foram selecionados 5 campos com dados para treinar o modelo para prever um resultado esperado pelo modelo de parendizado de máquina 

* Modelo 1- LM-H-PDNC

In [ ]:
# treinamento do modelo - fit modelo de regressão linear generalizado
fitglm<-glm("HorasUnit ~ Poder.Lin + Depart + Prod.Nome + Item.Cod",data=trainSample) # Adjusted R-squared:  0.235 
sumfitln<-summary(fitglm) #  resumo estatístico
sumfitln
#sumfitln$adj.r.squared
sumfitln$aic

A função cbind() cria matrizes com objetivo em unir linhas e colunas. Antes de executar essa função é necessário criar vetores para poder juntar. 

predict.glm(): obtém previsões e estima erros padrão dessas previsões de um objeto de modelo linear generalizado ajustado.

se.fit: interruptor lógico indicando se os erros padrão são necessários.

postResample(): calcula o desempenho entre reamostras

In [95]:
#teste do modelo
# predict.glm() - Método de previsão para ajustes GLM 
predglm<-predict.glm(fitglm, newdata = testSample, se.fit = TRUE) # se.fit = TRUE, lista com componentes de erros padrão estimados.
#obtenho estatisticas
res <- predglm$fit-testSample$HorasUnit
rmse <- sqrt(mean(res^2)) # média das diferenças entre as predições e observações reais ao quadrado
pred <- tibble(pred=predglm$fit) # tibble é releitura do data.frame
#colnames(pred) <- c('pred')
resultTest<- cbind(testSample,pred) # Conecta vetores formando colunas de uma matriz com cada vetor (c = column)
resultTest<-resultTest %>% ungroup()
resultTest<-resultTest[order(resultTest$Poder.Z, -resultTest$pred),]

Métricas de regressão usadas em todos os modelos

RMSE - Root Mean Squared Error: média das diferenças entre as predições e observações reais ao quadrado

R2 -  uma métrica de erro de regressão que justifica o desempenho do modelo

MAE - Mean Absolute Error: calcula o residual de cada ponto, onde valores residuais negativos e positivos não se anulam

In [ ]:
fit <- lm(HorasUnit ~ Poder.Lin, data = resultTest)
fig <- plot_ly(resultTest, x = ~Poder.Lin, y = ~pmax(pred,0), type = 'scatter', alpha = 0.65, mode = 'markers', name = 'Pred')
fig <- fig %>% add_trace(data = resultTest, x = ~Poder.Lin, y = fitted(fit), name = 'Regression Fit', mode = 'lines', alpha = 1)
fig
metricas<-postResample(pred = pmax(predglm$fit,0), obs = testSample$HorasUnit) # pred e obs são vetores de dados numéricos (pode ser um fator)
metricas
resumo<-tibble(Num=c(1),Nome=c("LM-H-PLDNC"),
               Log=c(0),Poder.Lin=c(1),Depart=c(1),Prod.Nome=c(1),Item.Cod=c(1),
               RMSE=c(metricas[1]),R2=c(metricas[2]),MAE=c(metricas[3]))
previsoes<-tibble(pred = pmax(predglm$fit,0), obs = testSample$HorasUnit, model=1) 

* Modelo 2 - LM-H-PZDNC

In [97]:
# treinamento do modelo - fit modelo de regressão linear generalizado
fitglm<-glm("HorasUnit ~ Poder.Z + Depart + Prod.Nome + Item.Cod",data=trainSample) # Adjusted R-squared:  0.235 
sumfitln<-summary(fitglm) # resumo estatístico

Assim como a função cbind(), rbind() cria matrizes com objetivo em unir linhas (row) e colunas (column). Antes de executar essa função é necessário criar vetores para poder juntar. 

In [ ]:
#teste do modelo
# predict.glm() - Método de previsão para ajustes GLM 
predglm<-predict.glm(fitglm, newdata = testSample, se.fit = TRUE) # se.fit = TRUE, lista com componentes de erros padrão estimados
metricas<-postResample(pred = pmax(predglm$fit,0), obs = testSample$HorasUnit) # pred e obs são vetores de dados numéricos (pode ser um fator)
metricas

# criando matrizes
# rbind() - Conecta vetores formando linhas de uma matriz com cada vetor (r = row) 
resumo<-rbind(resumo,tibble(Num=c(2),Nome=c("LM-H-PZDNC"),
                            Log=c(0),Poder.Lin=c(0),Depart=c(1),Prod.Nome=c(1),Item.Cod=c(1),
                            RMSE=c(metricas[1]),R2=c(metricas[2]),MAE=c(metricas[3])))
previsoes<-rbind(previsoes,tibble(pred = pmax(predglm$fit,0), obs = testSample$HorasUnit, model=2))

* Modelo 3 - LM-H-PZDN

In [100]:
# treinamento do modelo - fit modelo de regressão linear generalizado
fitglm<-glm("HorasUnit ~ Poder.Z + Depart + Prod.Nome",data=trainSample) # Adjusted R-squared:  0.235 
sumfitln<-summary(fitglm) # resumo estatístico

In [ ]:
#teste do modelo
# predict.glm() - Método de previsão para ajustes GLM 
numeroMod = 3
predglm<-predict.glm(fitglm, newdata = testSample, se.fit = TRUE) # se.fit = TRUE, lista com componentes de erros padrão estimados
metricas<-postResample(pred = pmax(predglm$fit,0), obs = testSample$HorasUnit) # pred e obs são vetores de dados numéricos (pode ser um fator)
metricas
resumo<-rbind(resumo,tibble(Num=c(numeroMod),Nome=c("LM-H-PZDN"),
                            Log=c(0),Poder.Lin=c(0),Depart=c(1),Prod.Nome=c(1),Item.Cod=c(0),
                            RMSE=c(metricas[1]),R2=c(metricas[2]),MAE=c(metricas[3])))
previsoes<-rbind(previsoes,tibble(pred = pmax(predglm$fit,0), obs = testSample$HorasUnit, model=numeroMod))

* Modelo 4 - LM-H-PZDC

In [102]:
# treinamento do modelo - fit modelo de regressão linear generalizado
fitglm<-glm("HorasUnit ~ Poder.Z + Depart + Item.Cod",data=trainSample) # Adjusted R-squared:  0.235 
sumfitln<-summary(fitglm) # resumo estatístico

In [ ]:
#teste do modelo
# predict.glm() - Método de previsão para ajustes GLM 
numeroMod = 4
predglm<-predict.glm(fitglm, newdata = testSample, se.fit = TRUE) # se.fit = TRUE, lista com componentes de erros padrão estimados.
metricas<-postResample(pred = pmax(predglm$fit,0), obs = testSample$HorasUnit) # pred e obs são vetores de dados numéricos (pode ser um fator)
metricas
resumo<-rbind(resumo,tibble(Num=c(numeroMod),Nome=c("LM-H-PZDC"),
                            Log=c(0),Poder.Lin=c(0),Depart=c(1),Prod.Nome=c(0),Item.Cod=c(1),
                            RMSE=c(metricas[1]),R2=c(metricas[2]),MAE=c(metricas[3])))
previsoes<-rbind(previsoes,tibble(pred = pmax(predglm$fit,0), obs = testSample$HorasUnit, model=numeroMod))

In [104]:
# converts an integer to the double class
sapply(mais_frequentes$HorasUnit_sca, as.double)

[1]  3.4843386125 -0.4527680193  0.0521804159  0.3895923986  0.6939649063
  [6] -0.4109570023  0.7115079904 -0.4475050941  0.4015801726 -0.3709002938
 [11]  4.1879624742  1.6018195045  1.2705476012 -0.1717862902  0.0898980465
 [16]  1.5086949668 -0.3910748404 -0.1472259725 -0.1788035238  1.0053546477
 [21] -0.4612471765  2.2684566974 -0.4624167155  0.2667908104 -0.1656462108
 [26] -0.1475183573 -0.3147624250 -0.3147624250  2.2839530883  3.7875415814
 [31]  2.7593706653  0.7404540790  0.0416545654  0.7138470682 -0.0001564515
 [36]  0.3711721603  2.0633488067 -0.4460431704 -0.3342060097 -0.4407802452
 [41] -0.4434117078 -0.3942910725 -0.3067218448 -0.4438502849 -0.4492594025
 [46] -0.4616857536  0.2115300957 -0.1866979116 -0.4437040925 -0.4282077016
 [51]  0.8600394351 -0.0302720791 -0.4562766361 -0.3820109137 -0.3489714388
 [56] -0.4486746330 -0.4510137109 -0.4454584009 -0.2703199454 -0.4472127093
 [61] -0.4480898635 -0.4606624071 -0.3092071150 -0.3992616130 -0.4556918666
 [66] -0.4130036955 -0.4650481781 -0.4048169229  0.7378226164  0.1793677750
 [71] -0.4349325505 -0.4314239337 -0.3165167334 -0.3548191335 -0.4439964773
 [76] -0.3741165259 -0.4656329475 -0.3761632190  1.4386688231 -0.4510137109
 [81] -0.3428313594 -0.3440008983 -0.4404878604 -0.3814261442 -0.2942954936
 [86] -0.4448736315 -0.4144656191 -0.4612471765 -0.0820241769 -0.4592004834
 [91] -0.4282077016 -0.4559842513 -0.4472127093 -0.3378608189 -0.3942910725
 [96] -0.4475050941 -0.4261610085 -0.3086223455 -0.4580309445  0.6138514893
[101] -0.4580309445 -0.4279153169 -0.4551070971 -0.4518908651 -0.3586201350
[106] -0.4045245382 -0.4630014849 -0.4615395613 -0.4027702298 -0.4606624071
[111] -0.1252971175 -0.2881554142 -0.4638786391 -0.4358097047 -0.4261610085
[116] -0.4600776376 -0.4589080987 -0.4565690208 -0.4299620100 -0.4627091002
[121] -0.4647557933 -0.4665101017  0.7983462563 -0.4564228284 -0.4005773443
[126] -0.4537913658 -0.4551070971 -0.4305467795 -0.3866890694 -0.4553994819
[131] -0.4580309445 -0.4643172162 -0.1592137466  2.1009202450  4.1289007580
[136]  0.5471877700  0.5682394709  2.1930214361 -0.4249914696  3.2982357295
[141] -0.4644634086 -0.2220764644 -0.4533527887  0.0015978569 -0.1717862902
[146] -0.3954606115 -0.1261742717 -0.1919608368  0.6778837460 -0.3980920741
[151] -0.2073110353  0.4088897910  1.2689394852  0.3036312868 -0.2267546201
[156] -0.4004311519  0.2469086485  0.0150475547  0.3121104441 -0.3536495945
[161] -0.3828880679  0.2843338944  0.1097802084 -0.4352249352 -0.4646096010
[166] -0.3016051119 -0.4584695216 -0.4496979796 -0.3928291489 -0.4666562941
[171] -0.4559842513  1.2421862821 -0.4650481781  0.3609386947 -0.4501365567
[176] -0.4492594025 -0.4656329475 -0.3612515976 -0.4498441719 -0.4609547918
[181] -0.4553994819 -0.4451660162 -0.4665101017 -0.4574461750  1.3570934824
[186] -0.4647557933 -0.4401954757 -0.4477974788 -0.4641710239 -0.4644634086
[191] -0.3369836647  0.7670610898 -0.4416573994 -0.4352249352 -0.4621243307
[196] -0.4653405628 -0.2773371790 -0.4431193230 -0.4650481781 -0.3121309623
[201] -0.4273305474 -0.4475050941 -0.3796718358 -0.4665101017 -0.4565690208
[206]  4.6458369672 -0.4647557933 -0.4498441719 -0.3264578143 -0.3355217410
[211] -0.4437040925 -0.3805489900 -0.4641710239 -0.4641710239 -0.4314239337
[216]  1.1582718635 -0.4644634086 -0.4466279399 -0.3252882754 -0.3378608189
[221] -0.4492594025 -0.3942910725 -0.4644634086 -0.4662177170 -0.4621243307
[226]  0.1919403186 -0.4650481781 -0.4624167155 -0.4594928681 -0.4665101017
[231] -0.4282077016 -0.4609547918 -0.4644634086 -0.4609547918 -0.4562766361
[236] -0.4668024865 -0.4647557933 -0.2568702477 -0.4668024865 -0.4510137109
[241] -0.4656329475 -0.3842037992 -0.4625629078 -0.4581771368 -0.4298158177
[246] -0.4666562941 -0.4613933689  0.3876918978 -0.4659253323  0.3390098396
[251]  2.4105556780  5.9817428155  0.8609165893  1.3085576166  4.3739191648
[256] -0.4165123123  5.8601107663 -0.4536451735 -0.4434117078 -0.4656329475
[261]  0.2024661690  0.2846262792  0.3866685512  0

* Modelo 5 - LM-LH-PLDNC

In [60]:
library('tidyverse') # manipulation, exploration and visualization of data

In [105]:
trainSample<-trainSample[trainSample$HorasUnit>0,]

função summary(): apresenta a distribuição das variáveis no conjunto de dados

In [ ]:
summary(trainSample$HorasUnit[trainSample$HorasUnit>0])

In [ ]:
summary(trainSample)

In [ ]:
# function used to return the number of rows of the specified matrix.
nrow(trainSample)

In [ ]:
summary(trainSample$HorasUnit_log)

In [29]:
# sapply - recebe uma lista, vetor ou quadro de dados como argumento e retorna um vetor ou matriz.
#mais_frequentes$HorasUnit_sca = sapply(mais_frequentes$HorasUnit_sca, as.double) #  Convertendo HorasUnit_sca  para a classe double 
#mais_frequentes

In [30]:
#filter(mais_frequentes,Poder.Lin<=0)

Tinham colunas NA, NaN e -Inf que estavm gerando erros e já foram retiradas, de 513 restaram 505 na tabela

In [35]:
#mais_frequentes = filter(mais_frequentes,HorasUnit_log != -Inf) # eliminando o - Inf

In [31]:
#mais_frequentes[mais_frequentes$Poder.Lin<0]

In [110]:
# treinamento do modelo - fit modelo de regressão linear generalizado
fitglm<-glm("HorasUnit_log ~ Poder.Lin + Depart + Prod.Nome + Item.Cod",data=trainSample) # Adjusted R-squared:  0.235 
sumfitln<-summary(fitglm) # resumo estatístico

In [ ]:
#teste do modelo
predglm<-predict.glm(fitglm, newdata = testSample, se.fit = TRUE) # se.fit = TRUE, lista com componentes de erros padrão estimados.
metricas<-postResample(pred = pmax(exp(predglm$fit),0), obs = testSample$HorasUnit) # pred e obs são vetores de dados numéricos (pode ser um fator)
metricas2<-postResample(pred = predglm$fit, obs = log(testSample$HorasUnit))
metricas
resumo<-rbind(resumo,tibble(Num=c(numeroMod),Nome=c("LM-LH-PLDNC"),
                            Log=c(1),Poder.Lin=c(1),Depart=c(1),Prod.Nome=c(1),Item.Cod=c(1),
                            RMSE=c(metricas[1]),R2=c(metricas2[2]),MAE=c(metricas[3])))
previsoes<-rbind(previsoes,tibble(pred = pmax(exp(predglm$fit),0), obs = testSample$HorasUnit, model=numeroMod))

* Modelo 6 - LM-LH-PZDNC

Treina o modelo com o log das horas unitárias

In [112]:
# treinamento do modelo - fit modelo de regressão linear generalizado
fitglm<-glm("HorasUnit_log ~ Poder.Z + Depart + Prod.Nome + Item.Cod",data=trainSample) # Adjusted R-squared:  0.235 
sumfitln<-summary(fitglm) # resumo estatístico

In [ ]:
#teste do modelo
predglm<-predict.glm(fitglm, newdata = testSample, se.fit = TRUE) # se.fit = TRUE, lista com componentes de erros padrão estimados.
metricas<-postResample(pred = pmax(exp(predglm$fit),0), obs = testSample$HorasUnit) # pred e obs são vetores de dados numéricos (pode ser um fator)
metricas2<-postResample(pred = predglm$fit, obs = log(testSample$HorasUnit))
metricas
resumo<-rbind(resumo,tibble(Num=c(numeroMod),Nome=c("LM-LH-PZDNC"),
                            Log=c(1),Poder.Lin=c(0),Depart=c(1),Prod.Nome=c(1),Item.Cod=c(1),
                            RMSE=c(metricas[1]),R2=c(metricas2[2]),MAE=c(metricas[3])))
previsoes<-rbind(previsoes,tibble(pred = pmax(exp(predglm$fit),0), obs = testSample$HorasUnit, model=numeroMod))

* Modelo  7 - GLM-H-PZDNC

In [ ]:
fitglm<-glm("HorasUnit ~ Poder.Lin + Depart + Prod.Nome + Item.Cod",data=trainSample,
            family = "poisson")  #  usada para encontrar a probabilidade de eventos que ocorrerem dentro de um determinado intervalo de tempo
sumfitln<-summary(fitglm) # resumo estatístico

In [ ]:
#teste do modelo
predglm<-predict.glm(fitglm, newdata = testSample, se.fit = TRUE) # se.fit = TRUE, lista com componentes de erros padrão estimados.
metricas<-postResample(pred = pmax(exp(predglm$fit),0), obs = testSample$HorasUnit) # pred e obs são vetores de dados numéricos (pode ser um fator)
metricas2<-postResample(pred = predglm$fit, obs = log(testSample$HorasUnit))
metricas
resumo<-rbind(resumo,tibble(Num=c(numeroMod),Nome=c("GLM-H-PZDNC"),
                            Log=c(0),Poder.Lin=c(1),Depart=c(1),Prod.Nome=c(1),Item.Cod=c(1),
                            RMSE=c(metricas[1]),R2=c(metricas2[2]),MAE=c(metricas[3])))
previsoes<-rbind(previsoes,tibble(pred = pmax(exp(predglm$fit),0), obs = testSample$HorasUnit, model=numeroMod))

* Modelo 8 - GLM-GH-PZDNC

In [ ]:
fitglm<-glm("HorasUnit ~ Poder.Lin + Depart + Prod.Nome + Item.Cod",data=trainSample,
            family = Gamma(link="log")) # Adjusted R-squared:  0.235 
sumfitln<-summary(fitglm) # resumo estatístico

In [ ]:
#teste do modelo
predglm<-predict.glm(fitglm, newdata = testSample, se.fit = TRUE) # se.fit = TRUE, lista com componentes de erros padrão estimados.
metricas<-postResample(pred = pmax(exp(predglm$fit),0), obs = testSample$HorasUnit) # pred e obs são vetores de dados numéricos (pode ser um fator)
metricas2<-postResample(pred = predglm$fit, obs = log(testSample$HorasUnit))
metricas
resumo<-rbind(resumo,tibble(Num=c(numeroMod),Nome=c("GLM-GH-PZDNC"),
                            Log=c(0),Poder.Lin=c(1),Depart=c(1),Prod.Nome=c(1),Item.Cod=c(1),
                            RMSE=c(metricas[1]),R2=c(metricas2[2]),MAE=c(metricas[3])))
previsoes<-rbind(previsoes,tibble(pred = pmax(exp(predglm$fit),0), obs = testSample$HorasUnit, model=numeroMod))

Ridge Regression - é uma extensão da regressão linear onde a função de perda é modificada para minimizar a complexidade do modelo. Essa modificação é feita adicionando um parâmetro de penalidade equivalente ao quadrado dos coeficientes.

SSE - Sum of Squares Error

SST - Sum of Squares Total 

In [119]:
# testar ridge
sampleDummies<- dummy_cols(mais_frequentes, remove_selected_columns = TRUE, remove_first_dummy = TRUE) #dummy_cols: cria variáveis ​​fictícias com base nos parâmetros fornecidos na função
sampleTudo<- mais_frequentes %>% select(-HorasUnit,-HorasUnit_log,-HorasUnit_sca,-linha)

traindummies <- sampleDummies[sampleDummies$linha %in% trainSample$linha,] #dummy_cols(trainSample, remove_selected_columns = TRUE, remove_first_dummy = TRUE)
testdummies <- sampleDummies[sampleDummies$linha %in% testSample$linha,] #dummy_cols(testSample, remove_selected_columns = TRUE, remove_first_dummy = TRUE)

library(glmnet) # ajusta a modelos lineares generalizados e similares via máxima verossimilhança penalizada
eval_results <- function(true, predicted, df) {
  SSE <- sum((predicted - true)^2) # A soma das diferenças quadradas entre os pontos de dados previstos e os pontos de dados observados
  SST <- sum((true - mean(true))^2) # A soma dos quadrados das diferenças entre os pontos de dados individuais e a média da variável de resposta
  R_square <- 1 - SSE / SST
  RMSE = sqrt(SSE/nrow(df))
  
  
  # Model performance metrics
  data.frame(
    RMSE = RMSE,
    Rsquare = R_square
  )
  
}

* Modelo 9 - RIDGET-H-PLDNC

model.matrix(): cria uma matriz de projeto (ou modelo)

In [120]:
# preparo modelo
x = model.matrix(as.formula('~ Poder.Lin + Depart + Prod.Nome + Item.Cod'),data=sampleTudo) # extrair fórmulas que foram incluídas em outros objetos
x <- x[,-1]
xTrain <- x[trainSample$linha,]
y_train = trainSample$HorasUnit

cv.glmnet(): Cross-validation for glmnet - Faz validação cruzada k-fold para glmnet, produz um gráfico e retorna um valor para lambda

In [ ]:
lambdas <- 10^seq(2, -3, by = -.1)
ridge_reg = cv.glmnet(xTrain, y_train, alpha = 0, lambda = lambdas) # treinamento

summary(ridge_reg) # resumo estatístico
optimal_lambda <- ridge_reg$lambda.min # melhor lambda
optimal_lambda

In [ ]:
xTest <- x[testSample$linha,]
y_test = testSample$HorasUnit
predictions_test <- predict(ridge_reg, s = optimal_lambda, newx = xTest) # previsao
metricas<-postResample(pred = pmax(predictions_test,0), obs = y_test) # metricas e output
metricas

In [123]:
resumo<-rbind(resumo,tibble(Num=c(numeroMod),Nome=c("RIDGET-H-PLDNC"),
                            Log=c(0),Poder.Lin=c(1),Depart=c(1),Prod.Nome=c(1),Item.Cod=c(1),
                            RMSE=c(metricas[1]),R2=c(metricas[2]),MAE=c(metricas[3])))
previsoes<-rbind(previsoes,tibble(pred = pmax(predictions_test,0), obs = testSample$HorasUnit, model=numeroMod))

previsto9<-tibble(pred = pmax(predictions_test,0))
previsto9<-cbind(previsto9,testSample)

previsto9$Item.Cod<-as.factor(paste(previsto9$Item.Cod))
previsto9$Prod.Nome<-as.factor(paste(previsto9$Prod.Nome))
previsto9$Depart<-as.factor(paste(previsto9$Depart))


ggp9<-previsto9 %>%
  ggplot(aes(x = HorasUnit,y = pred, colour= Depart,#shape=Prod.Nome)) +
             text =      paste(#"Depart:", Depart,
                               "Prod.Nome:", Prod.Nome,
                               "\nItem.Cod:", Item.Cod)))+
  geom_point()#+#geom_count()+
  #geom_smooth(method=lm,se=FALSE)

#ggplotly(ggp9)

* Modelo 10 - RIDGET-LH-PLDNC

In [124]:
# preparo do modelo
x = model.matrix(as.formula('~ Poder.Lin + Depart + Prod.Nome + Item.Cod'),data=sampleTudo)
x <- x[,-1]
xTrain <- x[trainSample$linha,]
y_train = trainSample$HorasUnit_log

In [ ]:
lambdas <- 10^seq(2, -3, by = -.1)
ridge_reg = cv.glmnet(xTrain, y_train, alpha = 0, lambda = lambdas) # treinamento

summary(ridge_reg) # resumo estatístico
optimal_lambda <- ridge_reg$lambda.min # melhor lambda
optimal_lambda

In [ ]:
xTest <- x[testSample$linha,]
y_test = testSample$HorasUnit_log
predictions_test <- predict(ridge_reg, s = optimal_lambda, newx = xTest) # previsão
eval_results(y_test, predictions_test, trainSample) # metricas e output


In [ ]:
metricas<-postResample(pred = pmax(exp(predictions_test),0), obs = testSample$HorasUnit)
metricas2<-postResample(pred = predictions_test, obs = testSample$HorasUnit_log)
metricas
resumo<-rbind(resumo,tibble(Num=c(numeroMod),Nome=c("RIDGET-LH-PLDNC"),
                            Log=c(1),Poder.Lin=c(1),Depart=c(1),Prod.Nome=c(1),Item.Cod=c(1),
                            RMSE=c(metricas[1]),R2=c(metricas2[2]),MAE=c(metricas[3])))
previsoes<-rbind(previsoes,tibble(pred = pmax(exp(predictions_test),0), obs = testSample$HorasUnit, model=numeroMod))

* Modelo 11 - RIDGETPoisson-H-PLDNC

In [127]:
# preparo do modelo
x = model.matrix(as.formula('~ Poder.Lin + Depart + Prod.Nome + Item.Cod'),data=sampleTudo)
x <- x[,-1]
xTrain <- x[trainSample$linha,]
y_train = trainSample$HorasUnit

In [ ]:
lambdas <- 10^seq(2, -3, by = -.1)
ridge_reg = cv.glmnet(xTrain, y_train, alpha = 0, lambda = lambdas, family = "poisson") # treinamento

summary(ridge_reg) # resumo estatístico 
optimal_lambda <- ridge_reg$lambda.min # melhor lambda
optimal_lambda

xTest <- x[testSample$linha,]
y_test = testSample$HorasUnit

In [ ]:
predictions_test <- predict(ridge_reg, s = optimal_lambda, newx = xTest)

metricas<-postResample(pred = pmax(exp(predictions_test),0), obs = y_test)
metricas2<-postResample(pred = predictions_test, obs = testSample$HorasUnit_log)
metricas


In [ ]:
resumo<-rbind(resumo,tibble(Num=c(numeroMod),Nome=c("RIDGETPoisson-H-PLDNC"),
                            Log=c(0),Poder.Lin=c(1),Depart=c(1),Prod.Nome=c(1),Item.Cod=c(1),
                            RMSE=c(metricas[1]),R2=c(metricas2[2]),MAE=c(metricas[3])))
previsoes<-rbind(previsoes,tibble(pred = pmax(exp(predictions_test),0), obs = testSample$HorasUnit, model=numeroMod))
previsoes

Final results

In [131]:
#exportacao dos resultados para csv
write.csv(resumo,"Resumo-regressoes.csv")

In [ ]:
previsoes$model<-as.factor(previsoes$model)
ggplot(previsoes,aes(x = obs,y = pred, colour= model)) +
  geom_count()+geom_smooth(method=lm,se=FALSE) # contar pontos sobrepostos + adicionar linhas de regressão ao diagrama de dispersão


In [91]:
ggp1<-previsoes %>% filter(model==9) %>%
  ggplot(aes(x = obs,y = pred, colour= model)) +
  geom_count()+geom_smooth(method=lm,se=FALSE)

#ggplotly(ggp1)

In [92]:
ggp<-previsoes %>% filter(model!=8) %>% ggplot(aes(x = obs,y = pred, colour= model)) +
  geom_count()+geom_smooth(method=lm,se=FALSE)
#ggplotly(ggp)